In [4]:
import copy
import click
import utility


def run_simulation(grid, R, thresholds, centers, max_steps):
    """
    Do the simulation.

    Inputs:
      grid (list of lists of ints): the grid
      R (int): neighborhood radius
      thresholds (float, float, float): the language
        state transition thresholds (A, B, C)
      centers (list of tuples): a list of community centers in the
        region
      max_steps (int): maximum number of steps

    Returns: the frequency of each language state (int, int, int)
    """


    def if_in_com_center(home_loc, com_center):
        '''
        Judge whether the given home is within the community 
        center's service distance (when there is at least one
        community center)

        Inputs:
            home_loc (tuple): the location of the given home
            com_center (tuple with (tuple, int) as elements):
                the location of the community center(tuple),
                the service distance (int)
            
        Returns: whether the given home is within the community 
            center's service distance (bool)
        '''

        i_home_loc = home_loc[0]
        j_home_loc = home_loc[1]
        i_center_home = com_center[0][0]
        j_center_home = com_center[0][1]
        r = com_center[1]
        if_i_in_cen = (i_home_loc >= i_center_home - r) and (
            i_home_loc <= i_center_home + r)
        if_j_in_cen = (j_home_loc >= j_center_home - r) and (
            j_home_loc <= j_center_home + r)
        if if_i_in_cen and if_j_in_cen:
            return True
        else:
            return False

        

    def engage_level(R, home_loc, grid):
        '''
        Calculate the engagement level of a given home
        
        Inputs:
            R (int): neighborhood radius
            home_loc (tuple): the home's location
            grid (list of lists of ints): the grid
        
        Returns: the engagement level of the given home (float)
        '''

        sum_lang_prefer = 0
        n_grid = len(grid)
        low_i = home_loc[0] - R
        up_i = home_loc[0] + R
        low_j = home_loc[1] - R
        up_j = home_loc[1] + R
        low_i = max(low_i, 0)
        low_j = max(low_j, 0)
        up_i = min(up_i, n_grid - 1)
        up_j = min(up_j, n_grid - 1)
       
        for i in range(low_i, up_i + 1):
            for j in range(low_j, up_j + 1):
                sum_lang_prefer += grid[i][j]
        total_homes = (up_i - low_i + 1) * (up_j - low_j +1)
        return sum_lang_prefer / total_homes

        
    def transmission_home(R, home_loc, grid, centers, thresholds): 
        '''
        Determine the language state of the next generation in a given
        home according to the transmission rules
        
        Inputs:
            R (int): neighborhood radius
            home_loc (tuple): the home's location
            grid (list of lists of ints): the grid
            centers (list of tuples): a list of community centers in the
                region
            thresholds (float, float, float): the language
                state transition thresholds (A, B, C)

        Return:the language state of the next generation in the given
            home (int)
        '''

        if centers == []:
            if_in_community_center = False
        else:
            for com_center in centers:
                if_in_community_center = if_in_com_center(home_loc, com_center)
                if if_in_community_center:
                    break

        i_home = home_loc[0]
        j_home = home_loc[1]
        status_home = grid[i_home][j_home]
        thresh_a, thresh_b, thresh_c = thresholds[0], thresholds[1], thresholds[2]
        engagement_level = engage_level(R, home_loc, grid)

        if status_home == 0:
            if engagement_level <= thresh_b:
                next_stage = 0
            else:
                next_stage = 1
        elif status_home == 1:
            if engagement_level < thresh_b:
                if if_in_community_center:
                    next_stage = 1
                else:
                    next_stage = 0
            elif engagement_level <= thresh_c:
                next_stage = 1
            else:
                next_stage = 2
        else:
            if if_in_community_center:
                next_stage = 2
            elif engagement_level >= thresh_b:
                next_stage = 2
            elif engagement_level > thresh_a:
                next_stage = 1
            else:
                next_stage = 0

        return next_stage
    
    now_steps = 0
    n_grid = len(grid)
    stop_signal = True
    next_gen = [[0 for _ in range(n_grid)] for _ in range(n_grid)]

    while True:
        for i in range(n_grid):
            for j in range(n_grid):
                home_loc = (i,j)
                transmitted = transmission_home(R, home_loc, grid, centers, thresholds)
                if grid[i][j] != transmitted and stop_signal:
                    stop_signal = False
                next_gen[i][j] = transmitted
        for i in range(n_grid):
            for j in range(n_grid):
                grid[i][j] = next_gen[i][j]
        now_steps += 1
        if now_steps >= max_steps or stop_signal:
            break
    
    count_0, count_1, count_2 = 0, 0, 0

    for row in grid:
        print()
        for status in row:
            print(status, end = ' ')
            if status == 0:
                count_0 += 1
            elif status == 1:
                count_1 += 1
            else:
                count_2 += 1
    
    return tuple([count_0, count_1, count_2])


grid = [
    [0, 0, 1, 1, 0],
    [0, 1, 1, 2, 0],
    [0, 0, 2, 2, 1],
    [1, 2, 1, 1, 2],
    [0, 1, 1, 0, 2]
]
R = 1
thresholds = (0.6, 0.8, 1.6) 
centers = []
max_steps = 1
run_simulation(grid,R,thresholds,centers,max_steps)


0 0 1 1 0 
0 0 1 2 1 
0 1 2 2 1 
0 2 1 1 2 
1 1 1 1 2 

(7, 12, 6)

In [5]:
grid = [
    [0, 0, 1, 1, 0],
    [0, 1, 1, 2, 0],
    [0, 0, 2, 2, 1],
    [1, 2, 1, 1, 2],
    [0, 1, 1, 0, 2]
]
R = 1
thresholds = (0.6, 0.8, 1.6) 
centers = []
max_steps = 1
run_simulation(grid,R,thresholds,centers,max_steps)


0 0 1 1 0 
0 0 1 2 1 
0 1 2 2 1 
0 2 1 1 2 
1 1 1 1 2 

(7, 12, 6)